# Organización de Info para creación de tablero

In [1]:
import os
import json
import sqlite3
import pandas as pd
import datetime as dt

In [2]:
### Realizamos el cambio de directoroi de trabajo al "Directorio Base" que se
current_dir = os.getcwd()
base_path = os.path.dirname(current_dir)

os.chdir(base_path)

In [3]:
import scripts.funciones as funciones

C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Cargamos el modelo entrenado 

In [5]:
version = 'ver010'    
mod_version = funciones.carga_model(base_path, f'models/{version}', version)
mod = mod_version['model'].steps[0][1]
model_sel = mod_version['model'].steps[1][1]

### Cargamos datos para realizar la predicción

In [6]:
d_ini = dt.datetime(2019,8,1)
d_fin = dt.datetime(2020,1,1)

In [7]:
data = funciones.read_clima_accidentes(d_ini, d_fin, poblado = True)
data_org = funciones.organizar_data_infoClima(data)

### agregamos la informacion relacionada a la cantidad de accidentes ocurridas
### en las ultimas X horas

### En caso que se considere acumulado de fallas, realiza la validacion
### si el modelo entrenado tiene la frecuencia utilizada
try:
    freq1 = mod.freq1
    freq2 = mod.freq2
except Exception as e:
    logger.info(f'Problemas con las frecuencias de las senales {e}')
    freq1 = '1H'
    freq2 = '1H'

d_ini_acc = d_ini - dt.timedelta(hours = int(freq2.replace('H', '')))
raw_accidentes = funciones.read_accidentes(d_ini_acc, d_fin)

### Agrega senal a corto plazo
data_org = funciones.obtener_accidentes_acumulados(data_org, 
                                                    raw_accidentes, 
                                                    freq = freq1)

### Agrega senal a largo plazo
data_org = funciones.obtener_accidentes_acumulados(data_org, 
                                                    raw_accidentes, 
                                                    freq = freq2)

data_org['poblado'] = data_org['BARRIO']
data_org= pd.get_dummies(data_org, columns=['poblado'])

In [8]:
X = data_org.drop(columns = ['TW','BARRIO','Accidente','summary'])
Y = data_org['Accidente']

### Realizamos la predicción de las fallas

In [9]:
preds_ff = mod.predict(X, model_sel)

In [10]:
info_preds = data_org[['TW','BARRIO']]
info_preds['Prob'] = preds_ff['Predicted']
info_preds['Accidente'] = Y

C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
info_preds

,TW,BARRIO,Prob,Accidente
0,2019-08-01 01:00:00,alejandria,0.306706,0.0
1,2019-08-01 02:00:00,alejandria,0.280793,0.0
2,2019-08-01 03:00:00,alejandria,0.364105,0.0
3,2019-08-01 04:00:00,alejandria,0.321523,0.0
4,2019-08-01 05:00:00,alejandria,0.321523,0.0
...,...,...,...,...
80757,2019-12-31 19:00:00,villacarlota,0.646678,0.0
80758,2019-12-31 20:00:00,villacarlota,0.682218,0.0
80759,2019-12-31 21:00:00,villacarlota,0.755662,0.0
80760,2019-12-31 22:00:00,villacarlota,0.777257,0.0


In [12]:
clima = data.drop(columns = ['Accidente']).merge(info_preds, how = 'left', on = ['TW','BARRIO']).dropna()

### Importamos la informacion cruda de los accidentes 

In [13]:
poblado = ['alejandria','altosdelpoblado',
        'astorga','barriocolombia',
        'castropol','elcastillo',
        'eldiamanteno2','elpoblado',
        'eltesoro','laaguacatala',
        'laflorida','lalinde',
        'laslomasno1','laslomasno2',
        'losbalsosno1','losbalsosno2',
        'losnaranjos','manila',
        'patiobonito','sanlucas',
        'santamariadelosangeles',
        'villacarlota']



def clean_barrio(string):
    to_rename = {'alejandria':'Alejandria',
                'altosdelpoblado':'Altos del Poblado',
                'astorga':'Astorga',
                'barriocolombia':'Barrio Colombia',
                'castropol':'Castropol',
                'elcastillo':'El Castillo',
                'eldiamanteno2':'El Diamante No.2',
                'elpoblado':'El Poblado',
                'eltesoro':'El Tesoro',
                'laaguacatala':'La Aguacatala',
                'laflorida':'La Florida',
                'lalinde':'Lalinde',
                'laslomasno1':'Las Lomas No.1',
                'laslomasno2':'Las Lomas No.2',
                'losbalsosno1':'Los Balsos No.1',
                'losbalsosno2':'Los Balsos No.2',
                'losnaranjos':'Los Naranjos',
                'manila': 'Manila',
                'patiobonito':'Patio Bonito',
                'sanlucas':'San Lucas',
                'santamariadelosangeles':'Santa Maria de los Angeles',
                'villacarlota':'Villa Carlota'}
    
    for key in to_rename.keys():
        string = string.replace(key,to_rename[key])
    
    return string

def clean_diaSem(string):
    to_rename = {'LUNES':'Lunes',
                'MARTES':'Martes',
                'MIERCOLES':'Miercoles',
                'JUEVES':'Jueves',
                'VIERNES':'Viernes',
                'SABADO':'Sabado',
                'DOMINGO':'Domingo'}
    
    for key in to_rename.keys():
        string = string.replace(key,to_rename[key])
    
    return string

In [14]:

d_ini_nic = dt.datetime(2017,1,1)
d_fin_nic = dt.datetime(2020,1,1)

accidentes = funciones.read_accidentes(d_ini_nic, d_fin_nic)
accidentes = accidentes[accidentes['BARRIO'].isin(poblado)]

In [15]:
accidentes.columns

Index(['Lon', 'Lat', 'OBJECTID', 'RADICADO', 'HORA', 'Dia_sem', 'PERIODO',
       'CLASE', 'DIRECCION', 'DIRECCION_ENC', 'CBML', 'TIPO_GEOCOD',
       'GRAVEDAD', 'BARRIO', 'COMUNA', 'DISENO', 'Mes', 'Dia', 'FECHA',
       'MES_NOMBRE', 'Hora_num', 'TW'],
      dtype='object')

In [16]:
cols = ['TW','BARRIO', 'Lon','Lat','CLASE','DISENO','GRAVEDAD','Dia_sem']
info_accidentes = accidentes[cols]
info_accidentes = info_accidentes.merge(info_preds, how = 'left', on = ['TW','BARRIO']).fillna(0)
info_accidentes = info_accidentes.merge(data.drop(columns = ['Accidente','hora','dia_sem']), how = 'left', on = ['TW','BARRIO'])

In [17]:
info_accidentes

,TW,BARRIO,Lon,Lat,CLASE,DISENO,GRAVEDAD,Dia_sem,Prob,Accidente,...,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,cloudCover,uvIndex,visibility
0,2017-04-12 11:00:00,villacarlota,-75.575113,6.224149,Choque,Tramo de via,SOLO DAÑOS,MIERCOLES,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-06-23 07:00:00,laaguacatala,-75.580131,6.194983,Choque,Interseccion,SOLO DAÑOS,VIERNES,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-08 18:00:00,manila,-75.576195,6.213677,Choque,Glorieta,SOLO DAÑOS,SABADO,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-05-13 19:00:00,villacarlota,-75.575010,6.225723,Choque,Tramo de via,SOLO DAÑOS,SABADO,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-14 15:00:00,laaguacatala,-75.580131,6.194983,Choque,Glorieta,HERIDO,VIERNES,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11083,2019-05-11 00:00:00,laaguacatala,-75.580022,6.194920,Choque,Tramo de via,HERIDO,SABADO,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11084,2019-03-11 18:00:00,villacarlota,-75.576948,6.218829,Choque,Tramo de via,SOLO DAÑOS,LUNES,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11085,2019-01-27 18:00:00,villacarlota,-75.574746,6.220077,Choque,Lote o Predio,SOLO DAÑOS,DOMINGO,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11086,2019-01-28 22:00:00,manila,-75.576192,6.213372,Caida Ocupante,Tramo de via,HERIDO,LUNES,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
clima['BARRIO'] = clima['BARRIO'].apply(lambda x: clean_barrio(x))
clima['DIA'] = clima['TW'].apply(lambda x: x.strftime("%Y-%m-%d"))

info_accidentes['BARRIO'] = info_accidentes['BARRIO'].apply(lambda x: clean_barrio(x))
info_accidentes['Dia_sem'] = info_accidentes['Dia_sem'].apply(lambda x: clean_diaSem(x))
info_accidentes['DIA'] = info_accidentes['TW'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [19]:
info_accidentes.columns

Index(['TW', 'BARRIO', 'Lon', 'Lat', 'CLASE', 'DISENO', 'GRAVEDAD', 'Dia_sem',
       'Prob', 'Accidente', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'windSpeed', 'cloudCover', 'uvIndex', 'visibility', 'DIA'],
      dtype='object')

In [20]:
info_accidentes['Hora_int'] = info_accidentes['TW'].apply(lambda x: f'[{x.hour},{x.hour+1})')
info_accidentes['Hora_intNic'] = info_accidentes['TW'].apply(lambda x: f'[{x.hour}:00,{x.hour+1}:00)')

In [21]:
info_accidentes['Hora_int']

0        [11,12)
1          [7,8)
2        [18,19)
3        [19,20)
4        [15,16)
          ...   
11083      [0,1)
11084    [18,19)
11085    [18,19)
11086    [22,23)
11087    [14,15)
Name: Hora_int, Length: 11088, dtype: object

In [22]:
info_accidentes['Accidente'] = 1

### Remueve puntos outliers
info_accidentes = info_accidentes[~((info_accidentes['BARRIO']=='La Florida')&(info_accidentes['Lon']<-75.64)&(info_accidentes['Lat']<6.18))].reset_index(drop = True)

In [23]:
db_file = 'visualizacion.sqlite3'
conn = sqlite3.connect(f'data/{db_file}')

In [24]:
clima.to_sql('clima', conn, if_exists = 'replace', index = False)
info_accidentes.to_sql('accidentes', conn, if_exists = 'replace', index = False)

In [25]:
info_accidentes.columns

Index(['TW', 'BARRIO', 'Lon', 'Lat', 'CLASE', 'DISENO', 'GRAVEDAD', 'Dia_sem',
       'Prob', 'Accidente', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'windSpeed', 'cloudCover', 'uvIndex', 'visibility', 'DIA',
       'Hora_int', 'Hora_intNic'],
      dtype='object')